# Anomaly detection

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
import math
import tensorflow as tf
import keras
from keras import optimizers, Sequential
from keras.models import Model
from keras.utils import plot_model
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint, TensorBoard
import numpy as np
from numpy import arange, sin, pi, random
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import scipy.integrate as integrate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.neighbors.kde import KernelDensity
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr

np.random.seed(1234)  
PYTHONHASHSEED = 0

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline

## Read Train and Test Data

In [ ]:
def gen_wave():
    """ Generate a synthetic data
    :return: the final data
    """
    t = np.arange(0.0, 100.0, 0.01)
    wave1 = sin(2 * 2 * pi * t)
    noise = random.normal(0, 0.2, len(t))
    wave1 = wave1 + noise
    print("wave1", len(wave1))
    wave2 = sin(2 * pi * t)
    print("wave2", len(wave2))
    t_rider = arange(0.0, 5.0, 0.01)
    wave3 = -2*sin(10 * pi * t_rider)
    print("wave3", len(wave3))
    insert = round(0.8 * len(t))
    wave1[insert:insert + 500] = wave1[insert:insert + 500] + wave3
    return wave1 - 2*wave2

In [ ]:
def z_norm(result):
    result_mean = result.mean()
    result_std = result.std()
    result -= result_mean
    result /= result_std
    return result, result_mean

In [ ]:
def get_split_prep_data(train_start, train_end,
                          test_start, test_end):
    data = gen_wave()
    print("Length of Data", len(data))

    # train data
    print ("Creating train data...")

    result = []
    for index in range(train_start, train_end - sequence_length):
        result.append(data[index: index + sequence_length])
    result = np.array(result)  # shape (samples, sequence_length)
    result, result_mean = z_norm(result)

    print ("Mean of train data : ", result_mean)
    print ("Train data shape  : ", result.shape)

    train = result[train_start:train_end, :]
    np.random.shuffle(train)  # shuffles in-place
    X_train = train[:, :-1]
    y_train = train[:, -1]

    # test data
    print ("Creating test data...")

    result = []
    for index in range(test_start, test_end - sequence_length):
        result.append(data[index: index + sequence_length])
    result = np.array(result)  # shape (samples, sequence_length)
    result, result_mean = z_norm(result)

    print ("Mean of test data : ", result_mean)
    print ("Test data shape  : ", result.shape)

    X_test = result[:, :-1]
    y_test = result[:, -1]

    print("Shape X_train", np.shape(X_train))
    print("Shape X_test", np.shape(X_test))

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    return X_train, y_train, X_test, y_test

In [ ]:
def flatten(X):
    '''
    Flatten a 3D array.
    
    Input
    X            A 3D array for lstm, where the array is sample x sequence length x features.
    
    Output
    flattened_X  A 2D array, sample x features.
    '''
    flattened_X = np.empty((X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)           
            


In [ ]:
def kde_sklearn(x, bandwidth=0.2, **kwargs):
	x_grid = np.linspace(x.min() - 0.9*x.min(), x.max() + x.max(), 500)
	"""Kernel Density Estimation with Scikit-learn"""
	kde_skl = KernelDensity(bandwidth=bandwidth, **kwargs)
	kde_skl.fit(x[:, np.newaxis])
	# score_samples() returns the log-likelihood of the samples
	log_pdf = kde_skl.score_samples(x_grid[:, np.newaxis])
	return np.exp(log_pdf), x_grid
   
def FindThreshold(x,h,p):
    tau=0
    x.sort() 
    for i in range(len(x)):
        int_K = integrate.quad(lambda s: (1/(h*np.sqrt(2*np.pi)))*np.exp(-0.5*(s-p)/h), (i-1)/len(x), i/len(x))
        tau=tau+int_K[0]*x[i]
    return tau

In [ ]:
# train on first 7000 samples and test on next 3000 samples (has anomaly)
sequence_length = 11
DATA_SPLIT_PCT = 0.2
SEED = 123 #used to help randomly select the data points
batch_size = 50
epochs = 3
X_train,y_train, X_test, y_test = get_split_prep_data(0, 6999, 7000, 10000)
X_train_X, X_valid = train_test_split(X_train, test_size=DATA_SPLIT_PCT, random_state=SEED)
timesteps =  X_train.shape[1] # equal to the sequence_length
n_features =  X_train.shape[2] # 1

In [ ]:
%%time
import keras
results = {}
for num_cells in [16, 32, 64,80,96]:
    for lr in [1e-4, 1e-3, 1e-2]:
            print('Running with', num_cells, 
                  'LSTM cells
                  'and learning rate =', lr, '...')

            # build network
            lstm_autoencoder = Sequential()
            # Encoder
            lstm_autoencoder.add(LSTM(100, activation='relu', input_shape=(timesteps, n_features), return_sequences=True))
            lstm_autoencoder.add(LSTM(25, activation='relu', return_sequences=False))
            lstm_autoencoder.add(RepeatVector(timesteps))
            # Decoder
            lstm_autoencoder.add(LSTM(25, activation='relu', return_sequences=True))
            lstm_autoencoder.add(LSTM(100, activation='relu', return_sequences=True))
            lstm_autoencoder.add(TimeDistributed(Dense(n_features)))
            lstm_autoencoder.summary()
            adam = optimizers.Adam(lr)
            lstm_autoencoder.compile(loss='mse', optimizer=adam)
            cp = ModelCheckpoint(filepath="lstm_autoencoder_classifier.h5",
                               save_best_only=True,
                               verbose=0)
            tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=True)
            lstm_autoencoder_history = lstm_autoencoder.fit(X_train, X_train, 
                                                epochs=epochs, 
                                                batch_size=batch_size, 
                                                validation_data=(X_valid, X_valid),
                                                verbose=2, callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')]).history
            print("Predicting...")
            predicted_train = lstm_autoencoder.predict(X_train)
            predicted = lstm_autoencoder.predict(X_test)
            mse = np.mean(np.power(flatten(X_test) - flatten(predicted), 2), axis=1)
            mse_train = np.mean(np.power(flatten(X_train) - flatten(predicted_train), 2), axis=1)
            #KDE
            # use grid search cross-validation to optimize the bandwidth
            params = {'bandwidth': np.linspace(0, 0.5, 10)}
            grid = GridSearchCV(KernelDensity(), params, cv = 20)
            grid.fit(mse_train[:, None])
            h=grid.best_estimator_.bandwidth
            tau=FindThreshold(mse_train,h,0.56)
            y_test1=np.ones(X_test.shape[0])
            y_test1[999:1499]=-1
            y_scores=np.ones(X_test.shape[0])
            y_scores[(mse-tau)>0]=-1
            accuracy_kde = accuracy_score(y_test1, y_scores)
            #OCSVM
            e=X_train - predicted_train
            nsamples, nx, ny = e.shape
            d2_e = e.reshape((nsamples,nx*ny))
            from sklearn import svm
            clf = svm.OneClassSVM(nu=0.0055, kernel="rbf", gamma=1.5)
            clf.fit(d2_e)
            e_t=X_test - predicted
            nsamples, nx, ny = e_t.shape
            d2_e_t = e_t.reshape((nsamples,nx*ny))
            y_scores = clf.predict(d2_e_t)
            accuracy_svm = accuracy_score(y_test1, y_scores)
            accuracy= min(accuracy_svm,accuracy_kde)
            results[(num_cells, lr)] = {'accuracy': accuracy}       
val_results = {key: results[key]['accuracy'] for key in results.keys()}
num_cells, lr = min(val_results, key=val_results.get)
print('Best parameters:', num_cells, 
        'and learning rate =', lr)

#Best parameters: num_cells=64,  lr=0.01
#num_cells=64
#lr = 0.01

## Generate Labels for Train Data

In [ ]:
%%time
import keras
lstm_autoencoder = Sequential()

# Encoder
lstm_autoencoder.add(LSTM(num_cells*4, activation='relu', input_shape=(timesteps, n_features), return_sequences=True))
lstm_autoencoder.add(LSTM(num_cells, activation='relu', return_sequences=False))
lstm_autoencoder.add(RepeatVector(timesteps))
# Decoder
lstm_autoencoder.add(LSTM(num_cells, activation='relu', return_sequences=True))
lstm_autoencoder.add(LSTM(num_cells*4, activation='relu', return_sequences=True))
lstm_autoencoder.add(TimeDistributed(Dense(n_features)))

lstm_autoencoder.summary()

adam = optimizers.Adam(lr)
lstm_autoencoder.compile(loss='mse', optimizer=adam)

cp = ModelCheckpoint(filepath="lstm_autoencoder_classifier.h5",
                               save_best_only=True,
                               verbose=0)

tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=True)


lstm_autoencoder_history = lstm_autoencoder.fit(X_train, X_train, 
                                                epochs=epochs, 
                                                batch_size=batch_size, 
                                                validation_data=(X_valid, X_valid),
                                                verbose=2, callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')]).history
                                                
plt.plot(lstm_autoencoder_history['loss'], linewidth=2, label='Train')
plt.plot(lstm_autoencoder_history['val_loss'], linewidth=2, label='Valid')
plt.legend(loc='upper right')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()


## Normalize Train and Test Data

In [ ]:
%%time
print("Predicting...")
predicted_train = lstm_autoencoder.predict(X_train)
predicted = lstm_autoencoder.predict(X_test)
#print("Reshaping predicted")
#predicted_train = np.reshape(predicted_train, (predicted_train.size,))
#predicted = np.reshape(predicted, (predicted.size,))





plt.figure()
plt.title("Actual Test Signal w/Anomalies")
plt.plot(y_train[:len(y_train)], 'b')




plt.figure()
plt.title("Squared Error")
mse = np.mean(np.power(flatten(X_test) - flatten(predicted), 2), axis=1)
plt.plot(mse, 'r')
mse_train = np.mean(np.power(flatten(X_train) - flatten(predicted_train), 2), axis=1)

import matplotlib.pyplot as plt
%matplotlib inline
plt.figure()
plt.title("Actual Test Signal with Anomalies")
plt.plot(y_test[:len(y_test)], 'b')
plt.savefig('Epoch.png', dpi=1200)
plt.show()

In [ ]:
#KDE
def kde_sklearn(x, bandwidth=0.2, **kwargs):
	x_grid = np.linspace(x.min() - 0.9*x.min(), x.max() + x.max(), 500)
	"""Kernel Density Estimation with Scikit-learn"""
	kde_skl = KernelDensity(bandwidth=bandwidth, **kwargs)
	kde_skl.fit(x[:, np.newaxis])
	# score_samples() returns the log-likelihood of the samples
	log_pdf = kde_skl.score_samples(x_grid[:, np.newaxis])
	return np.exp(log_pdf), x_grid
   
def FindThreshold(x,h,p):
    tau=0
    x.sort() 
    for i in range(len(x)):
        int_K = integrate.quad(lambda s: (1/(h*np.sqrt(2*np.pi)))*np.exp(-0.5*(s-p)/h), (i-1)/len(x), i/len(x))
        tau=tau+int_K[0]*x[i]
    return tau

#Plot histogram
fig, ax2 = plt.subplots(figsize = (9,7))
ax2.hist(mse_train, bins = 100, alpha = 0.5, density = True)

#Plot KDE
pdf, mse_grid = kde_sklearn(mse, bandwidth = 0.5)
pdf2, mse_grid2 = kde_sklearn(mse, bandwidth = 0.1)
pdf3, mse_grid3 = kde_sklearn(mse, bandwidth = 0.9)
ax2.plot(mse_grid, pdf, alpha = 0.9, color = 'green', linewidth = 2.0)
ax2.plot(mse_grid2, pdf2, alpha = 0.9, color = 'red', linewidth = 2.0)
ax2.plot(mse_grid3, pdf3, alpha = 0.9, color = 'yellow', linewidth = 2.0)
plt.show()

# use grid search cross-validation to optimize the bandwidth
params = {'bandwidth': np.linspace(0, 0.5, 10)}
grid = GridSearchCV(KernelDensity(), params, cv = 20)
grid.fit(mse_train[:, None])

print("best bandwidth: {0}".format(grid.best_estimator_.bandwidth))
h=grid.best_estimator_.bandwidth
tau=FindThreshold(mse_train,h,0.56)

y_test1=np.ones(X_test.shape[0])
y_test1[999:1499]=-1
y_scores=np.ones(X_test.shape[0])
y_scores[(mse-tau)>0]=-1
precision = precision_score(y_test1, y_scores)
recall    = recall_score(y_test1, y_scores)
accuracy = accuracy_score(y_test1, y_scores)
f1 = f1_score(y_test1, y_scores, average='macro')
print ('Tau : ', tau)
print ('Precision : ', precision)
print ('Recall: ', recall)
print ('Accuracy : ', accuracy)
print ('F1_score: ', f1)
fig=plt.figure(4)
red_dot, white_cross =plt.plot(mse, 'r--', np.ones(len(mse))*tau, 'b--')
plt.xlabel('t')
plt.ylabel('Anomaly scores')
plt.legend([red_dot, (red_dot, white_cross)], ["Anomaly scores", "Anomaly threshold"])
plt.show()
fig.savefig('plot.png', dpi=1200)  




OCSVM

In [ ]:
%%time
#OCSVM
e=X_train - predicted_train
nsamples, nx, ny = e.shape
d2_e = e.reshape((nsamples,nx*ny))

from sklearn import svm
clf = svm.OneClassSVM(nu=0.0055, kernel="rbf", gamma=1.5)
clf.fit(d2_e)

e_t=X_test - predicted
nsamples, nx, ny = e_t.shape
d2_e_t = e_t.reshape((nsamples,nx*ny))
y_scores = clf.predict(d2_e_t)
precision = precision_score(y_test1, y_scores)
recall    = recall_score(y_test1, y_scores)
accuracy = accuracy_score(y_test1, y_scores)
f1 = f1_score(y_test1, y_scores, average='macro')
print ('Precision : ', precision)
print ('Recall: ', recall)
print ('Accuracy : ', accuracy)
print ('F1_score: ', f1)


In [ ]:
#Tran et al., 2019

def build_model():
    model = Sequential()
    layers = {'input': 1, 'hidden1': 64, 'hidden2': 256, 'hidden3': 100, 'output': 1}

    model.add(LSTM(
            input_length=sequence_length - 1,
            input_dim=layers['input'],
            output_dim=layers['hidden1'],
            return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
            layers['hidden2'],
            return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
            layers['hidden3'],
            return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(
            output_dim=layers['output']))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop")
    print ("Compilation Time : ", time.time() - start)
    return model

#KDE
def kde_sklearn(x, bandwidth=0.2, **kwargs):
	x_grid = np.linspace(x.min() - 0.9*x.min(), x.max() + x.max(), 500)
	"""Kernel Density Estimation with Scikit-learn"""
	kde_skl = KernelDensity(bandwidth=bandwidth, **kwargs)
	kde_skl.fit(x[:, np.newaxis])
	# score_samples() returns the log-likelihood of the samples
	log_pdf = kde_skl.score_samples(x_grid[:, np.newaxis])
	return np.exp(log_pdf), x_grid
   
def FindThreshold(x,h,p):
    tau=0
    x.sort() 
    for i in range(len(x)):
        int_K = integrate.quad(lambda s: (1/(h*np.sqrt(2*np.pi)))*np.exp(-0.5*(s-p)/h), (i-1)/len(x), i/len(x))
        tau=tau+int_K[0]*x[i]
    return tau
  
def run_network(model=None, data=None):
    global_start_time = time.time()

    if data is None:
        print ('Loading data... ')
        # train on first 700 samples and test on next 300 samples (has anomaly)
        X_train,y_train, X_test, y_test = get_split_prep_data(0, 6999, 7000, 10000)
    else:
        X_train, y_train, X_test, y_test = data

    print ('\nData Loaded. Compiling...\n')

    if model is None:
        model = build_model()

    try:
        print("Training...")
        model.fit(
                X_train, y_train,
                batch_size=batch_size, nb_epoch=epochs, validation_split=0.05)
        print("Predicting...")
        predicted_train = model.predict(X_train)
        predicted = model.predict(X_test)
        print("Reshaping predicted")
        predicted_train = np.reshape(predicted_train, (predicted_train.size,))
        predicted = np.reshape(predicted, (predicted.size,))
    except KeyboardInterrupt:
        print("prediction exception")
        print ('Training duration (s) : ', time.time() - global_start_time)
        return model, y_test, y_train, 0

    try:
        mse = ((y_test - predicted) ** 2)
        plt.plot(mse, 'r')
        mse_train = ((y_train - predicted_train) ** 2)
        # use grid search cross-validation to optimize the bandwidth
        params = {'bandwidth': np.linspace(0, 0.5, 10)}
        grid = GridSearchCV(KernelDensity(), params, cv = 20)
        grid.fit(mse_train[:, None])
        print("best bandwidth: {0}".format(grid.best_estimator_.bandwidth))
        h=grid.best_estimator_.bandwidth
        tau=FindThreshold(mse_train,h,0.56)
        #tau=max(mse_train)
        y_test1=np.ones(len(y_test))
        y_test1[999:1499]=-1
        y_scores=np.ones(len(y_test))
        y_scores[(mse-tau)>0]=-1
        
        precision = precision_score(y_test1, y_scores)
        recall    = recall_score(y_test1, y_scores)
        accuracy = accuracy_score(y_test1, y_scores)
        f1 = f1_score(y_test1, y_scores, average='macro') 
     
        fig=plt.figure(3)
        red_dot, white_cross =plt.plot(mse, 'r--', np.ones(len(mse))*tau, 'b--')
        plt.xlabel('t')
        plt.ylabel('Anomaly scores')
        plt.legend([red_dot, (red_dot, white_cross)], ["Anomaly scores", "Anomaly threshold"])
        plt.show()
        fig.savefig('plot.png', dpi=1200)
        print ('Tau : ', tau)
        print ('Precision : ', precision)
        print ('Recall: ', recall)
        print ('Accuracy : ', accuracy)
        print ('F1_score: ', f1)
        
    except Exception as e:
        print("plotting exception")
        print (str(e))
        print ('Training duration (s) : ', time.time() - global_start_time)

    
    
    return model, y_test, predicted, y_train, predicted_train

run_network()